In [1]:
# from plotly import __version__
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# print(__version__) # requires version >= 1.9.0
# init_notebook_mode(connected=True)
# iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])

In [3]:
import os
import math
import random
import numpy as np
from time import time
from collections import Counter
from reconstruction.utils import binvox_rw
from reconstruction.utils.plot import plot_vol
from pprint import pprint

# Check datasets

In [4]:
path = '/home/rhermoza/data/3d/modelnet/ModelNet40/'
labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
files = [os.path.join(path,l,'train',ll) for l in labels
         for ll in os.listdir(os.path.join(path, l, 'train'))
         if ll[-4:] == '.off']

print(labels)
pprint(Counter([f.split('ModelNet40/')[1].split('/')[0] for f in files]))

['lamp', 'tv_stand', 'keyboard', 'sink', 'airplane', 'flower_pot', 'person', 'glass_box', 'guitar', 'bookshelf', 'stool', 'tent', 'bathtub', 'plant', 'door', 'bottle', 'dresser', 'cone', 'chair', 'bench', 'night_stand', 'wardrobe', 'laptop', 'cup', 'vase', 'piano', 'toilet', 'monitor', 'table', 'range_hood', 'bed', 'curtain', 'mantel', 'sofa', 'bowl', 'stairs', 'radio', 'car', 'xbox', 'desk']
Counter({'chair': 889,
         'sofa': 680,
         'airplane': 626,
         'bookshelf': 572,
         'bed': 515,
         'vase': 475,
         'monitor': 465,
         'table': 392,
         'toilet': 344,
         'bottle': 335,
         'mantel': 284,
         'tv_stand': 267,
         'plant': 240,
         'piano': 231,
         'dresser': 200,
         'night_stand': 200,
         'desk': 200,
         'car': 197,
         'bench': 173,
         'glass_box': 171,
         'cone': 167,
         'tent': 163,
         'guitar': 155,
         'flower_pot': 149,
         'laptop': 149,
    

In [5]:
path = '/home/rhermoza/data/3d/modelnet/ModelNet10/'
labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
files = [os.path.join(path,l,'train',ll) for l in labels
         for ll in os.listdir(os.path.join(path, l, 'train'))
         if ll[-4:] == '.off']

print(labels)
pprint(Counter([f.split('ModelNet10/')[1].split('/')[0] for f in files]))

['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor', 'table', 'bed', 'sofa', 'desk']
Counter({'chair': 889,
         'sofa': 680,
         'bed': 515,
         'monitor': 465,
         'table': 392,
         'toilet': 344,
         'dresser': 200,
         'night_stand': 200,
         'desk': 200,
         'bathtub': 106})


# read / convert / plot

In [13]:
# generate file
file = random.choice(files)

def check_fix_file(file):
    with open(file) as f:
        l1 = f.readline()
        l2 = f.readlines()

    if l1 != 'OFF\n' and l1[:3] == 'OFF':
        out = 'OFF\n'
        out += l1.split('OFF')[1]
        out += ''.join(l2)
        with open(file, 'w') as f:
            f.write(out)

def voxels_from_file(file, voxsize):
    cmd = f'tools/binvox/binvox -d {voxsize} -cb -e {file}'
    check_fix_file(file)
    out_file = file.split('.')[0] + '.binvox'
            
    if os.path.exists(out_file):
        os.remove(out_file)

    t = os.system(cmd)
    
    if t == 0:
        with open(out_file, 'rb') as f:
            d = binvox_rw.read_as_3d_array(f).data
        
        os.remove(out_file)
        return 1, d
    else:
        return 0, None

voxels = voxels_from_file(file, 32)
print('\nPlotting:', file)
plot_vol(voxels[1])


Plotting: /home/rhermoza/data/3d/modelnet/ModelNet10/bed/train/bed_0334.off


# Convert all ModelNet

In [17]:
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

def multithreading(func, args, workers):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        res = executor.map(func, args)
    return list(res)

get_label = lambda x: x.split('ModelNet')[1][3:].split('/')[0]

def get_voxels(files, voxsize):
    data = np.ndarray((0, *[voxsize]*3), dtype=np.bool)
    labels = []
    errors = []
    
    for i, file in enumerate(files):
        res = voxels_from_file(file, voxsize)
        if res[0] == 1:
            labels.append(get_label(file))
            data = np.vstack([data, res[1].reshape((1, *res[1].shape))])
        else:
            errors.append(file)

    return labels, data, errors

get_voxels_parallel = lambda x: get_voxels(*x)

def convert_all(path, voxsize):
    out_file = os.path.join(path, 'voxels.npy')
    labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    train_files = [os.path.join(path,l,'train',ll) for l in labels
                   for ll in os.listdir(os.path.join(path, l, 'train'))
                   if ll[-4:] == '.off']
    test_files = [os.path.join(path,l,'test',ll) for l in labels
                   for ll in os.listdir(os.path.join(path, l, 'test'))
                   if ll[-4:] == '.off']

    print(f'train: {len(train_files)}, test: {len(test_files)}')
    
    n_cpu = multiprocessing.cpu_count()
    output = {}
    
    for data_files, data_name in zip([train_files, test_files], ['train', 'test']):
        t0 = time()
        print(f'Launching {n_cpu} threads for {data_name} set...', end='')
        thread_size = math.ceil(len(data_files) / n_cpu)
        args = [(data_files[i*thread_size:(i+1)*thread_size], voxsize) for i in range(n_cpu)]
        res = multithreading(get_voxels_parallel, args, n_cpu)
        labels = []
        data = np.ndarray((0, *[voxsize]*3), dtype=np.bool)
        errors = []

        for l, d, e in res:
            labels += l
            data = np.vstack([data, d])
            errors += e
            
        output[data_name] = {'labels': labels, 'data': data, 'errors': errors}
        
        print('(%.2fs)' % (time() - t0))
    
    np.save(out_file, output)    
    print('\nSaved on: %s (%.2fM)' % (out_file, os.path.getsize(out_file) / 1024**2))

In [19]:
convert_all('/home/rhermoza/data/3d/modelnet/ModelNet10/', voxsize=32)

train: 3991, test: 908
Launching 8 threads for train set...(99.45s)
Launching 8 threads for test set...(20.24s)

Saved on: /home/rhermoza/data/3d/modelnet/ModelNet10/voxels.npy (153.17M)


In [22]:
convert_all('/home/rhermoza/data/3d/modelnet/ModelNet40/', voxsize=32)

train: 9843, test: 2468
Launching 8 threads for train set...(1129.71s)
Launching 8 threads for test set...(138.28s)

Saved on: /home/rhermoza/data/3d/modelnet/ModelNet40/voxels.npy (384.91M)


In [4]:
modelnet10 = np.load('/home/rhermoza/data/3d/modelnet/ModelNet10/voxels.npy').item()

idx = random.choice(range(len(modelnet10['train']['labels'])))
print(modelnet10['train']['labels'][idx])
plot_vol(modelnet10['train']['data'][idx])

chair


In [19]:
modelnet40 = np.load('/home/rhermoza/data/3d/modelnet/ModelNet40/voxels.npy').item()

idx = random.choice(range(len(modelnet40['train']['labels'])))
print(modelnet40['train']['labels'][idx])
plot_vol(modelnet40['train']['data'][idx])

vase
